In [177]:
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from collections import namedtuple

Entry = namedtuple('Entry',['value','cluster'])

In [85]:
files =  listdir('./')

In [86]:
filelist = list()
for f in files:
    if f.endswith('.out'):
        filelist.append(f)

In [87]:
filelist

['tbirdOverlap1131523501-1131541501.out',
 'tbirdOverlap1131532501-1131550501.out',
 'tbirdOverlap1131541501-1131559501.out',
 'tbirdOverlap1131550501-1131568501.out',
 'tbirdOverlap1131559501-1131577501.out',
 'tbirdOverlap1131568501-1131586501.out',
 'tbirdOverlap1131577501-1131595501.out',
 'tbirdOverlap1131586501-1131604501.out',
 'tbirdOverlap1131595501-1131613501.out',
 'tbirdOverlap1131604501-1131622501.out',
 'tbirdOverlap1131613501-1131631501.out',
 'tbirdOverlap1131622501-1131640501.out',
 'tbirdOverlap1131631501-1131649501.out',
 'tbirdOverlap1131640501-1131658501.out',
 'tbirdOverlap1131649501-1131667501.out',
 'tbirdOverlap1131658501-1131676501.out',
 'tbirdOverlap1131667501-1131685501.out',
 'tbirdOverlap1131676501-1131694501.out',
 'tbirdOverlap1131685501-1131703501.out',
 'tbirdOverlap1131694501-1131712501.out']

In [88]:
documents = dict()
for f in filelist:
    a = open(f,'r')
    words = ""
    for w in a.readlines():
        words += w.lstrip().strip()
        words += ' '
    a.close()
    documents[f] = words.strip()

In [89]:
doc_set = list()
for d in documents.itervalues():
    doc_set.append(d)


In [90]:
texts = []
for d in doc_set:
    tokens = tokenizer.tokenize(d)
    texts.append(tokens)

In [91]:

dictionary = corpora.Dictionary(texts)

In [92]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [103]:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=200)

In [202]:
outData = list()
for x in(ldamodel.print_topics(num_topics=5, num_words=40)):
    parts =  x[1].split('+')
    sum = 0
    lines = list()
    empty = True
    # print x
    for p in parts:
        val,cluster = p.lstrip().strip().split('*')
        # print val,cluster
        
        if (sum + float(val) < .8) or empty:
            sum += float(val)
            # print 'adding:',cluster
            empty = False
            lines.append(Entry(float(val),int(cluster)))
    #print sum,lines
    #print
    outData.append((sum,lines))
            

In [173]:
import re
templateFile = '/Users/dgrossman/data/tbird.Clusters'
tf = open(templateFile,'r').readlines()
templateList = list()
for t in tf:
    # print t.strip()
    unescaped= re.sub(r'[\^]','',re.sub(r'[\\]', '', t)).strip()
    templateList.append(unescaped)


In [223]:
for topic, X in enumerate(outData):
    s = X[0]
    ent = X[1]
    print 'topic=%i|count=%i|energy=%f' % (topic, len(ent), s)
    wordBag = set()
    for e in ent:
        cluster, string = templateList[e.cluster].split(',',1)
        print '\t%5i| %1.4f| %s' % (int(cluster),e.value,string)
        for s in string.split():
            wordBag.add(s)
    print
    print 'words used : %s' % ' '.join(sorted(wordBag))
    print
    

topic=0|count=14|energy=0.798000
	    1| 0.1860| tbird admin INT FILEPATH rrd_update FILEPATH FILEPATH illegal attempt to update using time INT when last update time is INT minimum one second step
	  114| 0.0800| dn INT ntpd INT synchronized to IPADDR stratum INT
	  115| 0.0670| cn INT ntpd INT synchronized to IPADDR stratum INT
	   12| 0.0570| tbird admin INT FILEPATH data_thread got not answer from any thunderbird_a INT datasource
	   11| 0.0570| tbird admin INT FILEPATH data_thread got not answer from any thunderbird_b INT datasource
	  116| 0.0550| bn INT ntpd INT synchronized to IPADDR stratum INT
	   83| 0.0430| aadmin INT xinetd INT start rsync pid= INT from= IPADDR
	   14| 0.0410| tbird admin INT FILEPATH data_thread got not answer from any thunderbird_d INT datasource
	   13| 0.0410| tbird admin INT FILEPATH data_thread got not answer from any thunderbird_c INT datasource
	   84| 0.0410| badmin INT xinetd INT start rsync pid= INT from= IPADDR
	   85| 0.0370| cadmin INT xinetd 